In [23]:
import keras
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Flatten
from keras import optimizers
from keras.models import Sequential
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [29]:
# Import Dependencies
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import cv2
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization

In [2]:
# Set up Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
/content/drive/MyDrive/Data_analytics /group_project_4/pet_expressions_data

In [3]:
# Change Directory
%cd '/content/drive/MyDrive/Data_analytics /group_project_4/pet_expressions_data'

/content/drive/MyDrive/Data_analytics /group_project_4/pet_expressions_data


In [4]:
# Define the path to the dataset folders
happy_folder = "/content/drive/MyDrive/Data_analytics /group_project_4/pet_expressions_data/happy"
sad_folder = "/content/drive/MyDrive/Data_analytics /group_project_4/pet_expressions_data/sad"
angry_folder = "/content/drive/MyDrive/Data_analytics /group_project_4/pet_expressions_data/angry"
other_folder = "/content/drive/MyDrive/Data_analytics /group_project_4/pet_expressions_data/other"

In [ ]:
cv2.cvtColor(original_image_bgr, cv2.COLOR_BGR2RGB)

In [5]:
# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  # Resize to a fixed size for the model
            images.append(img)
    return images

In [6]:
# Load images and labels for each emotion
happy_images = load_images_from_folder(happy_folder)
sad_images = load_images_from_folder(sad_folder)
angry_images = load_images_from_folder(angry_folder)
other_images = load_images_from_folder(other_folder)

In [101]:
happy_images[1].shape

(224, 224, 3)

In [7]:
# Create labels for each emotion category
happy_labels = [0] * len(happy_images)
sad_labels = [1] * len(sad_images)
angry_labels = [2] * len(angry_images)
other_labels = [3] * len(other_images) # 3???? not sure if thats correct

In [8]:
# Concatenate images and labels
x = np.array(happy_images + sad_images + angry_images + other_images)
y = np.array(happy_labels + sad_labels + angry_labels + other_labels)

In [24]:
# Normalize pixel values to range [0, 1]
x = x.astype('float32') / 255.0

In [10]:
# One-hot encode the labels
y = tf.keras.utils.to_categorical(y, 4)

In [11]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

In [31]:
x_validate, y_validate, x_test, y_test= train_test_split(x_test,y_test, test_size=.50, random_state=100)

In [19]:
x_train.shape

(296, 224, 224, 3)

In [20]:
x_test.shape

(74, 224, 224, 3)

In [21]:
y_train.shape

(296, 4)

In [22]:
y_test.shape

(74, 4)

In [34]:
x_validate.shape

(37, 224, 224, 3)

In [35]:
IMAGE_SIZE = [224, 224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#here [3] denotes for RGB images(3 channels)

#don't train existing weights
for layer in vgg.layers:
 layer.trainable = False

x = Flatten()(vgg.output)
prediction = Dense(10, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [36]:
history = model.fit_generator(
                    x_train,
                    epochs = 18, verbose=5,
                    validation_data = y_validate,
                    validation_steps = x_validate)

<ipython-input-36-7f02d77dd92e>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(x_train,


ValueError: ignored

In [ ]:
imputs= x_train

In [18]:
vgghist = model.fit(x= x_train, y= y_train, steps_per_epoch = 100, epochs = 10)
# x_train.reshape(-1, 48, 48, 1), y_train, batch_size=32, epochs=100, validation_split=0.1, class_weight=class_weights,verbose=0)

Epoch 1/10


NotImplementedError: ignored

In [ ]:
# Compile the model with class weights
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

In [78]:
# Calculate class weights
total_samples = len(y_train)
class_weights = {0: total_samples / np.sum(y_train[:, 0]),
                 1: total_samples / np.sum(y_train[:, 1]),
                 2: total_samples / np.sum(y_train[:, 2]),
                 3: total_samples / np.sum(y_train[:, 3])}

In [ ]:
# Train/Fit the model with class weights
history = model.fit(x_train.reshape(-1, 48, 48, 1), y_train, batch_size=32, epochs=100, validation_split=0.1, class_weight=class_weights,verbose=0)

ValueError: ignored

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(x_test.reshape(-1, 48, 48, 1), y_test)
losstr, accuracytr = model.evaluate(x_train.reshape(-1, 48, 48, 1), y_train)

In [ ]:
print(f"Test accuracy: {accuracy*100:.2f}%")
print(f"Train accuracy: {accuracytr*100:.2f}%")

In [ ]:
# Save the trained model
model.save("pet_expressions_model.h5")

In [ ]:
##################################################################################

In [ ]:
from keras.models import load_model
# Load the saved model
loaded_model = load_model("pet_expressions_model.h5")

In [ ]:
# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images

In [ ]:
# Load a custom test image
custom_test_image_path = "/kaggle/input/pets-facial-expression-dataset/Angry/16924834.jpg"

custom_test_image = cv2.imread(custom_test_image_path)
custom_test_image = cv2.cvtColor(custom_test_image, cv2.COLOR_BGR2GRAY)
custom_test_image = cv2.resize(custom_test_image, (48, 48))
custom_test_image = custom_test_image.astype('float32') / 255.0

In [ ]:
# Reshape the image to match the model input shape
custom_test_image = np.expand_dims(custom_test_image, axis=0)
custom_test_image = np.expand_dims(custom_test_image, axis=-1)

In [ ]:
# Make predictions on the custom test image
prediction = loaded_model.predict(custom_test_image)
prediction_prob = prediction[0]

In [ ]:
emotion_label = np.argmax(prediction[0])

In [ ]:
# Map the predicted label to emotion class
emotion_classes = {0: 'happy', 1: 'sad', 2: 'angry'}
predicted_emotion = emotion_classes[emotion_label]

In [ ]:
# Print the custom test image and its predicted label
print(f"Predicted Emotion: {predicted_emotion}")
print(f"Confidence [happy, sad, angry]: {prediction_prob}")

In [ ]:
import matplotlib.pyplot as plt

#Display the custom test image using matplotlib
plt.imshow(custom_test_image[0, :, :, 0])
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
from PIL import Image
# Display the original custom test image using PIL
img_pil = Image.open(custom_test_image_path)
plt.imshow(np.array(img_pil))
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()